In [ ]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np

import random
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, Lambda
from keras.optimizers import RMSprop, Adam
from keras import backend as K
from keras.utils import to_categorical
from keras.applications.densenet import DenseNet121

import numpy as np
from keras.applications.densenet import DenseNet121, DenseNet169
import keras.backend as K
import cv2
import os
from keras.callbacks import LearningRateScheduler,ReduceLROnPlateau,CSVLogger,ModelCheckpoint
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score,roc_auc_score
from sklearn.metrics import cohen_kappa_score,classification_report,auc,roc_curve
from keras.callbacks import Callback
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
from keras.applications.vgg16 import VGG16
# from tqdm import tqdm

In [ ]:
num_classes = 2
epochs = 20

In [ ]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

In [ ]:
def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

In [ ]:
def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

In [ ]:
def create_pairs(x, digit_indices):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
    for d in range(num_classes):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, num_classes)
            dn = (d + inc) % num_classes
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)

In [ ]:
def create_pairs(x, digit_indices):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    n = max([len(digit_indices[d]) for d in range(num_classes)]) - 1
    print (n)
    d=0
    dn=1
#     for d in range(num_classes):
    for i in range(n):
        z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
        pairs += [[x[z1], x[z2]]]
#         inc = random.randrange(1, num_classes)
#         dn = (d + inc) % num_classes
        z1, z2 = digit_indices[d][i], digit_indices[dn][i%len(digit_indices[dn])]
        pairs += [[x[z1], x[z2]]]
        labels += [1, 0]
    return np.array(pairs), np.array(labels)

In [ ]:
# def create_base_network(input_shape):
#     '''Base network to be shared (eq. to feature extraction).
#     '''
#     input = Input(shape=input_shape)
#     x = Flatten()(input)
#     x = Dense(128, activation='relu')(x)
#     x = Dropout(0.1)(x)
#     x = Dense(128, activation='relu')(x)
#     x = Dropout(0.1)(x)
#     x = Dense(128, activation='relu')(x)
#     return Model(input, x)

def create_base_network(input_shape):
    dense_model = DenseNet121(weights='imagenet', include_top=False,input_shape=(224,224,3),pooling='avg')
    preds = Dense(1024,activation='relu')(dense_model.output)
    model = Model(dense_model.input,preds)
#     model=dense_model
    return model




In [ ]:
def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)


def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [ ]:
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std = np.array([0.229, 0.224, 0.225])

def img_standardization(x):
    
#     x=cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x=x.astype('float16')/255.
#     return x
    return ((x-imagenet_mean)/imagenet_std)

In [ ]:
x_fold1=np.load('../Pathology_NoPathology/Good_CSV_with_Good_Labels/x_fold1.npy', mmap_mode='r')
y_fold1=np.load('../Pathology_NoPathology/Good_CSV_with_Good_Labels/fold1_labels.npy')

x_fold2=np.load('../Pathology_NoPathology/Good_CSV_with_Good_Labels/x_fold2.npy', mmap_mode='r')
y_fold2=np.load('../Pathology_NoPathology/Good_CSV_with_Good_Labels/fold2_labels.npy')

x_fold3=np.load('../Pathology_NoPathology/Good_CSV_with_Good_Labels/x_fold3.npy', mmap_mode='r')
y_fold3=np.load('../Pathology_NoPathology/Good_CSV_with_Good_Labels/fold3_labels.npy')

In [ ]:
x_train=np.concatenate([x_fold2, x_fold3])
y_train=np.concatenate([y_fold2, y_fold3])

x_val=x_fold1
y_val=y_fold1

Y_train=to_categorical(y_train, num_classes=2)
Y_val=to_categorical(y_val, num_classes=2)

In [ ]:
X_train=np.zeros(x_train.shape,dtype='float32')
for index in range(x_train.shape[0]):
    X_train[index]=img_standardization(x_train[index])
    print (index)

X_val=np.zeros(x_val.shape,dtype='float32')
for index in range(x_val.shape[0]):
    X_val[index]=img_standardization(x_val[index])
    print (index)

In [ ]:
# create training+test positive and negative pairs
digit_indices = [np.where(y_train == i)[0] for i in range(num_classes)]
tr_pairs, tr_y = create_pairs(X_train, digit_indices)

digit_indices = [np.where(y_val == i)[0] for i in range(num_classes)]
te_pairs, te_y = create_pairs(X_val, digit_indices)

In [ ]:
input_shape=(224,224,3)
base_network = create_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model([input_a, input_b], distance)


# encoded_l = model(input_shape)
# encoded_r = model(input_shape)
    
#     # Add a customized layer to compute the absolute difference between the encodings
# L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
# L1_distance = L1_layer([encoded_l, encoded_r])

# # Add a dense layer with a sigmoid unit to generate the similarity score
# prediction = Dense(1,activation='sigmoid')(L1_distance)

# # Connect the inputs with the outputs
# model = Model(inputs=[left_input,right_input],outputs=prediction)
    

In [ ]:
model.summary()

In [ ]:
adam=Adam(lr=1e-3)
model.compile(loss=contrastive_loss, optimizer=adam, metrics=[accuracy])

model_checkpoint = ModelCheckpoint(
        os.path.join('./weights/', 'model1.hdf5'),
        monitor='val_loss', mode='min',save_best_only=True, verbose=1, save_weights_only=True)

callbacks=([model_checkpoint])


In [ ]:
model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y,
          batch_size=32,
          epochs=60,
          validation_data=([te_pairs[:, 0], te_pairs[:, 1]], te_y), verbose=1, callbacks=callbacks, shuffle=True)

In [ ]:
model.load_weights('./weights/model1.hdf5')

In [ ]:
y_train_pred = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]], verbose=1)
tr_acc = compute_accuracy(tr_y, y_train_pred)
y_pred = model.predict([te_pairs[:, 0], te_pairs[:, 1]], verbose=1)
te_acc = compute_accuracy(te_y, y_pred)

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

In [ ]:
print (y_train_pred, tr_y)

In [ ]:
Test_templates=[]
count=0
for idx, i in enumerate(y_train_pred):
    if np.around(i)==0 and tr_y[idx]==1:
        if i>=0.0 and i<=0.2:
            Test_templates.append(tr_pairs[idx, 0])
            Test_templates.append(tr_pairs[idx, 1])
templates=np.array(Test_templates)

In [ ]:
print (templates.shape)

In [ ]:
x_val=np.load('/opt/bucketdata/Users/Rohit/MURA_Dataset/bone_data/HAND/X_test.npy', mmap_mode='r')
X_val=np.zeros(x_val.shape,dtype='float32')
for index in range(x_val.shape[0]):
    X_val[index]=img_standardization(x_val[index])
    print (index)

In [ ]:
final_preds=[]
for idx, i in enumerate(X_val):
    preds=[]
    for jdx, j in enumerate(templates[0:1]):
        preds.append(model.predict([np.expand_dims(j, axis=0), np.expand_dims(i, axis=0)], verbose=0))
    print(idx)
    final_preds.append(np.average(preds))

In [ ]:
final_preds

In [ ]:
Y_pred=[]
for i in final_preds:
    if i>=0.8:
        Y_pred.append(1)
    else:
        Y_pred.append(0)

In [ ]:
np.sum(Y_pred)

In [ ]:
# Y_pred=np.argmax(op,axis=1)
Y_test=np.argmax(Y_val,axis=-1)
# Y_test=Y_val.copy()
target_names = ['normal','Pathology']

print("Confusion Matrix:")
print (confusion_matrix(Y_test,Y_pred))

print("Classification report:")
print(classification_report(Y_test,Y_pred,target_names=target_names))

print("Kappa score:")
print(cohen_kappa_score(Y_test,Y_pred))

# Converting Healthy+other into NonTB cases to calculate ROC
y_true=np.array(Y_test)
y_true[np.where(y_true==1)]=1
y_true[np.where(y_true==2)]=1

#Taking probabilities of TB class
preds = op[:,1]
fpr, tpr, threshold = roc_curve(y_true, preds)
roc_auc = auc(fpr, tpr)


# ROC curve plot
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.savefig('TB_ROC_curve.png',dpi=200)
plt.show()

In [ ]:
1) 1-way shot
Confusion Matrix:
[[ 235 1131]
 [   8   71]]
Precision: 0.06
Sensitivity: 0.9
Specificity: 0.17
    
2) 10-way shot:
Confusion Matrix:
[[ 289 1077]
 [   8   71]]

Precision: 0.06
Sensitivity: 0.9
Specificity: 0.21
